# Convert DICOM to NIFTI

Gather necessary pre-requisites:

In [1]:
# import everything you need
from nipype.interfaces.dcm2nii import Dcm2niix
from nipype import Node, Workflow
import os, re


200210-15:08:53,227 nipype.utils WARNING:
	 Could not check for version updates: 
Connection to server could not be made


In [2]:
# define useful functions

# get subject directories from parent directory
def get_subdirs(dir):
    subdirs=[]
    for i in os.listdir(dir):
        # verify subject number in directory and exclude other dirs
        match=re.match(r'(.*\d{3}$)', i)
        if os.path.isdir(os.path.join(dir, i)) and match:
            subdirs.append(os.path.join(dir, match.group(0)))
    return sorted(subdirs)

# get iterator to go over multiple objects at once
def grouped(iterable, n):
    "s -> (s0,s1,s2,...sn-1), (sn,sn+1,sn+2,...s2n-1), (s2n,s2n+1,s2n+2,...s3n-1), ..."
    return zip(*[iter(iterable)]*n)


In [3]:
# get subject dir-list
subdirs=get_subdirs("/fMRI/")
print(subdirs)

['/fMRI/FED006', '/fMRI/FED007', '/fMRI/FED008', '/fMRI/FED009', '/fMRI/FED010', '/fMRI/FED011', '/fMRI/FED012', '/fMRI/FED013', '/fMRI/FED014', '/fMRI/FED015', '/fMRI/FED016', '/fMRI/FED017', '/fMRI/FED018', '/fMRI/FED019', '/fMRI/FED020', '/fMRI/FED021', '/fMRI/FED022', '/fMRI/FED023', '/fMRI/FED024', '/fMRI/FED025', '/fMRI/FED026', '/fMRI/FED027', '/fMRI/FED028', '/fMRI/FED029', '/fMRI/FED030', '/fMRI/FED031', '/fMRI/FED032', '/fMRI/FED033', '/fMRI/FED034', '/fMRI/FED035', '/fMRI/FED036', '/fMRI/FED037', '/fMRI/FED038', '/fMRI/FED039', '/fMRI/FED040', '/fMRI/FED041', '/fMRI/FED042', '/fMRI/FED043', '/fMRI/FED044', '/fMRI/FED045', '/fMRI/FED046', '/fMRI/FED047', '/fMRI/FED048', '/fMRI/FED049', '/fMRI/FED050', '/fMRI/FED051', '/fMRI/FED052', '/fMRI/FED053', '/fMRI/FED054', '/fMRI/FED055', '/fMRI/FED056', '/fMRI/FED057', '/fMRI/FED058', '/fMRI/FED059', '/fMRI/FED060', '/fMRI/FED061', '/fMRI/FED062', '/fMRI/FED063', '/fMRI/FED064', '/fMRI/FED065', '/fMRI/FED066', '/fMRI/FED067', '/fMRI/

In [6]:
# get fMRI sequence dirs via subject lists
sequencedirs=[]
for i in subdirs:
    for j in os.listdir(i):
        if os.path.isdir(os.path.join(i, j)):
            match=re.match(r'((WIP840|FMRI).*\d{4}$)', j)
            if match:
                sequencedirs.append(os.path.join(i, match.group(0)))
                
# sort the list entries
sequencedirs=sorted(sequencedirs)

# extract the middle number of however many fMRI sequence dirs there are to select the functional runs (VARIATIONS. SOLVE.)
funcdirs=[]
for i in subdirs:
    dirs=[]
    for j in sequencedirs:
        if i.rsplit('/', 1)[1] == j.rsplit('/', 2)[1]:
            dirs.append(j)
    funcdirs.append(sorted(dirs)[len(dirs) // 2])

In [7]:
print(funcdirs)
print(len(funcdirs))

['/fMRI/FED006/WIP840_36SLICE_1800TR_FS_0009', '/fMRI/FED007/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED008/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED009/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED010/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED011/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED012/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED013/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED014/WIP840_36SLICE_1800TR_FS_0018', '/fMRI/FED015/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED016/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED017/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED018/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED019/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED020/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED021/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED022/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED023/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED024/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED025/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED026/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED027/WIP840_36SLICE_1800

Convert .IMA files to NIFTI

In [8]:
# use sequence-list as source_dir and save output of converter in parent dir

# iterate over sequence and subject dirs, create nodes and run them to create NIFTI files
for i,j in zip(funcdirs, subdirs):
    print(i,j)
    # create DICOM-to-NIFTI converter node
    converter=Node(Dcm2niix(), name='converter')
    converter.inputs.compress='n' # do not compress -> SPM options etc.
    converter.inputs.source_dir=i
    converter.inputs.output_dir=j
    converter.run()

/fMRI/FED006/WIP840_36SLICE_1800TR_FS_0009 /fMRI/FED006
200210-17:42:46,144 nipype.workflow INFO:
	 [Node] Setting-up "converter" in "/tmp/tmp8w7tp_jl/converter".
200210-17:42:46,148 nipype.workflow INFO:
	 [Node] Running "converter" ("nipype.interfaces.dcm2nii.Dcm2niix"), a CommandLine Interface with command:
dcm2niix -b y -z n -x n -t n -m n -o /fMRI/FED006 -s n -v n /fMRI/FED006/WIP840_36SLICE_1800TR_FS_0009
200210-17:42:47,307 nipype.interface INFO:
	 stdout 2020-02-10T17:42:47.307743:Chris Rorden's dcm2niiX version v1.0.20190902  (JP2:OpenJPEG) GCC6.3.0 (64-bit Linux)
200210-17:42:47,309 nipype.interface INFO:
	 stdout 2020-02-10T17:42:47.307743:Found 280 DICOM file(s)
200210-17:42:47,309 nipype.interface INFO:
	 stdout 2020-02-10T17:42:47.307743:Convert 280 DICOM as /fMRI/FED006/WIP840_36SLICE_1800TR_FS_0009_WIP840_36slice_1800TR_fs_20141106153429_9 (72x68x36x280)
200210-17:42:47,387 nipype.interface INFO:
	 stdout 2020-02-10T17:42:47.387840:Conversion required 1.111298 seconds (

For testing or re-doing single subjects:

In [14]:
# print sequencedirs + subdirs and crop, if testing
sequencedir=sequencedirs[43]
subdir=subdirs[43]

print(sequencedir, subdir)

/VBM/FED049/T1_MPRAGE_SAG_ISO_0_9_0009 /VBM/FED049


In [15]:
# use sequence-list as source_dir and save output of converter in parent dir

# insert the two variable values directly, create nodes and run them to create NIFTI files

# create DICOM-to-NIFTI converter node
converter=Node(Dcm2niix(), name='converter')
converter.inputs.compress='n' # do not compress -> SPM options etc.
converter.inputs.source_dir=sequencedir
converter.inputs.output_dir=subdir
converter.run()

200124-15:38:14,358 nipype.workflow INFO:
	 [Node] Setting-up "converter" in "/tmp/tmpjnxkds55/converter".
200124-15:38:14,362 nipype.workflow INFO:
	 [Node] Running "converter" ("nipype.interfaces.dcm2nii.Dcm2niix"), a CommandLine Interface with command:
dcm2niix -b y -z n -x n -t n -m n -o /VBM/FED049 -s n -v n /VBM/FED049/T1_MPRAGE_SAG_ISO_0_9_0009
200124-15:38:14,833 nipype.interface INFO:
	 stdout 2020-01-24T15:38:14.833851:Chris Rorden's dcm2niiX version v1.0.20190902  (JP2:OpenJPEG) GCC6.3.0 (64-bit Linux)
200124-15:38:14,835 nipype.interface INFO:
	 stdout 2020-01-24T15:38:14.833851:Found 192 DICOM file(s)
200124-15:38:14,836 nipype.interface INFO:
	 stdout 2020-01-24T15:38:14.833851:Convert 192 DICOM as /VBM/FED049/T1_MPRAGE_SAG_ISO_0_9_0009_t1_mprage_sag_iso_0.9_20151123115353_9 (256x256x192x1)
200124-15:38:15,102 nipype.interface INFO:
	 stdout 2020-01-24T15:38:15.102243:Conversion required 0.712450 seconds (0.546249 for core code).
200124-15:38:15,135 nipype.workflow INFO:
